# Script avec les Fonctions de Base

## Connexion a la Base de Données Neo4j

In [7]:
from neo4j import GraphDatabase

def execute_cypher_query(dbDriver, cypherQuery, queryParams={}) :
    """ Execute une requête cypher et retourne les enregistrements obtenus.
    
    Arguments:
    dbDriver -- the Neo4j driver
    cypherQuery -- the cypher query to execute
    queryParams -- the parameters to pass to the query

    Returns:
    The records returned by the query
    """
    res = dbDriver.execute_query(
        cypherQuery,
        queryParams,
    )
    return res.records

# Connexion à la base Neo4j
DB_URL = 'bolt://localhost:7687'
DB_USER = 'neo4j'
DB_PASS = 'neo4jpassword'

DB_DRIVER = GraphDatabase.driver(DB_URL, auth=(DB_USER, DB_PASS))

## Game Fonctions

In [6]:
def getNumber(text,choices=[]) :
    """
    Demander à l'utilisateur de saisir un chiffre.
    Si des choix sont spécifiés, l'utilisateur ne pourra entrer que l'un de ces choix.
    
    Arguments:
    text -- le texte à afficher à l'utilisateur
    choices -- les choix possibles

    Returns:
    Le chiffre saisi par l'utilisateur
    """
    while True:
        try:
            digit = int(input(text))
            if len(choices) == 0 or digit in choices:
                return digit
            else:
                print("Veuillez entrer un chiffre entre ",min(choices)," et ",max(choices),".", sep="")
        except ValueError:
            print("Veuillez entrer un chiffre valide.")

def getDifficulty(diff1, diff2) :
    print("Selection du niveau de difficulté souhaité :")
    print(" - 1 :", diff1)
    print(" - 2 :", diff2)
    return getNumber("Choisissez une difficulté (1 ou 2) : ",[1, 2])

def diff_factor(diff):
    return (2-diff)

## Fonctions Films

In [10]:
def getRandomFilms(diff, count=1) :
    factor = diff_factor(diff)
    params = {"budget": (factor * 200000000), "year": (factor * 2000), "count": count}
    query = '''
    match (m:Movie)
    where m.year >= $year and m.budget >= $budget
    return m.title as movieTitle, rand() as randomNum
    order by randomNum
    limit $count
    '''
    return execute_cypher_query(DB_DRIVER, query, params)


def getShortestMoviePath(actorName1,actorName2) :
    params = {"actorName1": actorName1, "actorName2": actorName2}
    query = '''
    MATCH (fa:Actor {name: $actorName1})
    MATCH (sa:Actor {name: $actorName2})
    MATCH path = shortestPath((fa)-[:ACTED_IN*]-(sa))
    UNWIND nodes(path) as n
    MATCH (n:Movie)
    RETURN n.title as movieTitle
    '''
    return execute_cypher_query(DB_DRIVER, query, params)

## Fonctions Actors

In [4]:
def getAllActors(movie) :
    params = {"movieTitle": movie}
    query = '''
    match (a:Actor)
    match (a)-[:ACTED_IN]->(m:Movie{title: $movieTitle})
    return a.name as actorName
    '''
    return execute_cypher_query(DB_DRIVER, query, params)

def getRandomActors(diff) :
    factor = diff_factor(diff)
    params = {"minMovieCount": (factor * 20)}
    query = '''
    match (a:Actor)
    match (a)-[:ACTED_IN]->(m)
    with a, count(m) as movies
    where movies >= $minMovieCount
    return a.name as actorName, rand() as randomNum
    order by randomNum
    limit 2
    '''
    return execute_cypher_query(DB_DRIVER, query, params)

def getRandomActorsFromMovie(movie) :
    params = {"movieTitle": movie}
    query = '''
    match (a:Actor)
    match (a)-[:ACTED_IN]->(m:Movie{title: $movieTitle})
    return a.name as actorName, rand() as randomNum
    order by randomNum
    limit 2
    '''
    actors = []
    return execute_cypher_query(DB_DRIVER, query, params)